# Ejercicio: Search Based Fuzzing

## Requerimientos

In [ ]:
from IPython.display import clear_output
!apt-get update
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz
!pip install fuzzingbook
clear_output()

## Función a validar

In [ ]:
def safety_check(temp, pressure, humidity):
    """
    Check if the machine's operational parameters are within a safe range.
    Returns True if the parameters are safe, otherwise False.
    """
    # Check temperature
    if temp >= 100:
        return False

    # Check pressure
    if pressure >= 200:
        return False

    # Check humidity
    if humidity < 20 or humidity > 70 :
        return False

    # Check combined condition for temperature and pressure
    if temp > 90 or pressure >= 180:
        return False

    # If all conditions are met, the parameters are safe
    return True

## Implementación del algoritmo Search Based

In [ ]:
### Cotas valores de los vecinos
MAX = 1000
MIN = 0

### 1) Defina la función neighbors:
Usted puede implementar la estrategia que usted prefiera para obtener los vecinos. Por ejemplo, sumando uno y restando uno a los valores recibidos de entrada.
Los valores de los vecinos no deben ser mayores a 1000, tampoco negativos, es decir que los valores deben variar entre 0 y 1000.

In [ ]:
def neighbors(x, y, z):
  #### Completa con tu implementación de la función aquí!
  pass

print(neighbors(10, 10, 10))


### 2) Defina la Fitness Function:
En este ejercicio utilizaremos el branch distance como fitness function.

Recuerde que el branch distance, es un valor que indica que tan cerca estan los valores de entrada en cumplir una condición.

Por ejemplo, considere esta condición `a > 100` con `a = 20`, esta condición deberia devolver `False`, y la distancia para que devuelva `True` es `81`, ya que si sumamos `80` a `20` tendriamos `101` y la condición seria `True`.

Por otro lado la distancia para que devuelva `True` es `0`, porque la condición ya evalua a `True`.

### 2.1) Implemente la función `evaluate_condition`;
En esta, cada vez que se evalúe una condición debe devolver el resultado de evaluar la condicion `(True, False)` de esta forma la función `instrumented_safety_check` funcionara igual que la original.

Además de devolver el valor, la función debe guardar el branch distance de cada
condición en una variable global (por ejemplo un diccionario). Note que la distancia que nos interesa en este ejercicio es la distancia para condiciones evaluen False.

In [ ]:
def evaluate_condition(id, op, a, b):
  #### Completa con tu implementación de la función aquí!
  pass

In [ ]:
def instrumented_safety_check(temp, pressure, humidity):
    """
    Check if the machine's operational parameters are within a safe range.
    Returns True if the parameters are safe, otherwise False.
    """
    # Check temperature
    if evaluate_condition(1, '>=',temp, 100):
        return False

    # Check pressure
    if evaluate_condition(2, '>=',pressure,200):
        return False

    # Check humidity
    if (evaluate_condition(3,'<',humidity,20) or evaluate_condition(4,'>',humidity, 70)) :
        return False

    # Check combined condition for temperature and pressure
    if evaluate_condition(5, '>',temp,90) or evaluate_condition(6,'>=',pressure,180):
        return False

    # If all conditions are met, the parameters are safe
    return True

### 2.2) Implemente la fitness function;

Dado 3 valores devuelve el branch distance total, que es la sumatoria de sumar las branch distance de todas las condiciones.

Para saber si fitness función esta bien, esta debe devolver 0 si los tres numeros ingresados como argumentos pasan todas las condiciones y la función `safety_check` devuelve verdad.

Si los valores esta cerca de cumplir las condiciones deben devolver un valor cercano a cero, si se aleja debe devolver un valor mas grande.

Puede probar esto, ejecutando la función con varios valores y ver si funciona. En la celda posterior a esta se ejemplifican dos casos.

In [ ]:
def normalize(x):
    return x/ MAX

In [ ]:
def get_fitness_validation(x,y,z):
    # ...

    try:
      # Ejecuta el método bajo prueba
      # Este método al ejecutarse ademas de devolverte el resultado deberia
      # almacenar los branch distance de cada función en una variable global
        instrumented_safety_check(x,y,z)
    except BaseException:
        pass

    #### Completa con tu implementación de la función aquí!
      # En esta parte debes sumar los branch distance almacenados en la variable global y sumarlos
      # antes de sumarlos debes normalizar el branch distance de cada condición utilizando la función normalize

    ####
    return fitness

In [ ]:
get_fitness_validation(20,20,79)


2.009

In [ ]:
instrumented_safety_check(894, 225, 24)

False

## Probemos tu solución

In [ ]:
import random

def hillclimb_validation(log=False):
    x,y,z = random.randint(MIN, MAX), random.randint(MIN, MAX),random.randint(MIN, MAX)
    fitness = get_fitness_validation(x,y,z)
    print("Initial input: %d %d %d at fitness %.4f" % (x,y,z, fitness))
    iterations = 0
    while fitness > 0:
        changed = False
        iterations += 1
        for (nextx,nexty,nextz) in neighbors(x,y,z):
            new_fitness = get_fitness_validation(nextx,nexty,nextz)
            if new_fitness < fitness:
                x,y,z = nextx,nexty,nextz
                fitness = new_fitness
                changed = True
                if log:
                  print("New value: %d, %d, %d at fitness %.4f" % (x, y, z, fitness))
                break

        if not changed:
            x, y, z = random.randint(MIN, MAX), random.randint(MIN, MAX),random.randint(MIN, MAX)
            fitness = get_fitness_validation(x,y,z)

    print("Found optimum after %d iterations at %d, %d, %d" % (iterations, x, y, z))
    return iterations

### Probando una iteración y mostrando logs (True)

*   Si su solución es "adecuada" la fitess function deberia reducir en cada iteración, y terminar en 0 cuando encuentre la solución



In [ ]:
hillclimb_validation(True)

### Probando 100 iteraciones, sin logs
*   Esto es sobre todo para ver cuantas iteraciones toma en promedio porque las iteraciones varias dependiendo el seed inicial que es randomico

In [ ]:
population = []
for i in range(100):
  population.append(hillclimb_validation())
result = sum(population)
print("average")
result/100.0